In [36]:
import requests
import json
import pandas as pd
from time import sleep

In [37]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Emilia Romagna
df_er = df_comuni[df_comuni['cod_regione'] == '08']

In [38]:
df_er.head(2)

,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune
3,080130130005,ALTO RENO TERME,CM,N,NaN,7,08,EMILIA-ROMAGNA,013,BOLOGNA,013,0005
4,080130130010,ANZOLA DELL'EMILIA,CM,N,NaN,7,08,EMILIA-ROMAGNA,013,BOLOGNA,013,0010


In [81]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_93458/3372484651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_93458/3372484651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
184,080560560220,MONCHIO DELLE CORTI,CM,N,NaN,7,08,EMILIA-ROMAGNA,056,PARMA,056,0220,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
260,080660660030,BAGNARA DI ROMAGNA,CM,N,NaN,7,08,EMILIA-ROMAGNA,066,RAVENNA,066,0030,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
133,080500500190,MARANELLO,CM,M,NaN,7,08,EMILIA-ROMAGNA,050,MODENA,050,0190,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...


In [82]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [86]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff'][:1]):
   print(f"\nProcessing record {idx + 1}")
   print(f"URL: {url}")

   response = requests.get(url, headers=headers)
   print(f"Response status code: {response.status_code}")
   
   data = response.json()

   comune = df_er.iloc[idx]['desc']
   provincia = df_er.iloc[idx]['provincia']
   regione = df_er.iloc[idx]['regione']
   url = df_er.iloc[idx]['url']

   print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

   # Start with basic comune info
   comune_info = {
       'comune': comune,
       'provincia': provincia,
       'regione': regione,
       'url': url
   }
   
   # Add candidate totals
   if 'cand' in data:
       for cand in data['cand']:
           nome_completo = cand['nome'] + " " + cand['cogn']
           comune_info[f"{nome_completo}_total"] = cand['voti']
           comune_info[f"{nome_completo}_perc"] = cand['perc']
           print(f"{nome_completo}: {cand['voti']} votes ({cand['perc']}%)")

   comuni_data.append(comune_info)
   
   print(f"Processed {comune}")
   sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())


Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/013/CM/0005
Response status code: 200
Processing data for Comune: ALTO RENO TERME, Provincia: BOLOGNA
ELENA UGOLINI: 942 votes (48,33%)
MICHELE DE PASCALE: 941 votes (48,28%)
FEDERICO SERRA: 49 votes (2,51%)
LUCA TEODORI: 17 votes (0,87%)
Processed ALTO RENO TERME

Creating DataFrame...

Sample of processed data:
            comune provincia         regione  \
0  ALTO RENO TERME   BOLOGNA  EMILIA-ROMAGNA   

                                                 url  ELENA UGOLINI_total  \
0  https://elezioni.interno.gov.it/risultati/2024...                  942   

  ELENA UGOLINI_perc  MICHELE DE PASCALE_total MICHELE DE PASCALE_perc  \
0              48,33                       941                   48,28   

   FEDERICO SERRA_total FEDERICO SERRA_perc  LUCA TEODORI_total  \
0                    49                2,51                  17   

  LUCA TEODORI_perc  
0              0,87  


In [87]:
df_data

,comune,provincia,regione,url,ELENA UGOLINI_total,ELENA UGOLINI_perc,MICHELE DE PASCALE_total,MICHELE DE PASCALE_perc,FEDERICO SERRA_total,FEDERICO SERRA_perc,LUCA TEODORI_total,LUCA TEODORI_perc
0,ALTO RENO TERME,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,942,"48,33",941,"48,28",49,"2,51",17,"0,87"


In [89]:
df_data['perc_diff'] = round(((df_data['MICHELE DE PASCALE_total'] - df_data['ELENA UGOLINI_total']) / df_data['ELENA UGOLINI_total']) * 100, 2)

In [91]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data.head(2)

,comune,provincia,regione,url,ELENA UGOLINI_total,ELENA UGOLINI_perc,MICHELE DE PASCALE_total,MICHELE DE PASCALE_perc,FEDERICO SERRA_total,FEDERICO SERRA_perc,LUCA TEODORI_total,LUCA TEODORI_perc,perc_diff
0,Alto Reno Terme,Bologna,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,942,"48,33",941,"48,28",49,"2,51",17,"0,87",-0.11


In [44]:
df_data.to_csv('../output/risultati_er.csv', index=False, encoding='UTF-8-sig')